In [1]:
"""Newest version as of 2016-06-28"""

from matplotlib import pyplot as plt
from IPython.display import clear_output
import os
import datetime
import numpy
import time
import warnings
import RPi.GPIO as GPIO
import subprocess
warnings.filterwarnings("ignore", module="numpy")
warnings.filterwarnings("ignore", module="GPIO")

started = datetime.datetime.now()
tstart = started.strftime("%Y-%m-%d_%H:%M:%S") # to start on the current time
#tstart = '2016-06-06_18:54:52' # start on a specific time

# acquisition and control system folders
dirroot = os.path.join(os.getcwd(),os.pardir)
dirbash = os.path.join(dirroot,'bash')
dirpy = os.path.join(dirroot,'python')
dirc = os.path.join(dirroot,'c')
dirskel = os.path.join(dirroot,'skeleton')
dirdata = os.path.join(dirroot,'data')
dirarchive = os.path.join(dirroot,'archive')

# folders for this experiment
direxp = os.path.join(dirdata,tstart)
dirin = os.path.join(direxp,'in')
dirbin = os.path.join(direxp,'bin')
dirout = os.path.join(direxp,'out')
dirplot = os.path.join(direxp,'plot')

# folders for incoming data
dirspectra = os.path.join(dirin,'spectra')
dirtemp = os.path.join(dirin,'temperature')
dirthermo = os.path.join(dirin,'thermograph')
dirosc = os.path.join(dirin,'oscilloscope')

#############
### SETUP ###
#############

# synchronize time on all machines with:
# sudo service ntp stop && sudo ntpdate time.nrc.ca && sudo service ntp start

def setup_gpio():
    ### SET UP RPI.GPIO CONTROL
    GPIO.cleanup()
    GPIO.setmode(GPIO.BOARD)
    GPIO.setup(18, GPIO.OUT)
    GPIO.output(18, GPIO.LOW) ## turn the camera on

def mkdirs():
    print "Setting up this experiment's directory structure..."
    
    #### copy the skeleton that will hold this experiment's data
    subprocess.Popen(['/bin/cp','-r',dirskel,direxp])
    
    #### copy the run scripts into position
    subprocess.Popen(['/bin/cp',os.path.join(dirbash,'run_oscilloscope.sh'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirbash,'run_spectroscopy.sh'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirbash,'run_thermometry.sh'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirbash,'stoprun.sh'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirbash,'run_spectroscopy.sh'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirc,'run_thermography'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirpy,'acquire_oscilloscope.py'),dirbin])
    subprocess.Popen(['/bin/cp',os.path.join(dirpy,'launch.ipynb'),dirbin])
    
    print "Structure complete:"
    print os.listdir(direxp)
    print "Experiment start time: %s" % (tstart)

########################
### CAPTURE THE DATA ###
########################

def capture_data():
    print "Capturing data..."
    run_thermometry()
    #run_thermography()
    run_spectroscopy()
    run_oscilloscope()

def run_thermometry():
    ### run thermometry logging - WORKS
    subprocess.Popen([os.path.join(dirbin,'run_thermometry.sh')],cwd=dirtemp)
    print "thermometry started."
    return True

def run_thermography():
    #### run thermography logging- WORKS
    subprocess.Popen([os.path.join(dirbin,'run_thermography.sh')],cwd=dirthermo)
    print "thermography started."
    return True
    
def run_spectroscopy():
    #### run spectroscopy logging - WORKS
    subprocess.Popen([os.path.join(dirbin,'run_spectroscopy.sh')],cwd=dirspectra)
    print "spectroscopy started."
    return True
    
def run_oscilloscope():
    ### run oscilloscope logging - WORKS
    subprocess.Popen([os.path.join(dirbin,'run_oscilloscope.sh')],cwd=dirbin)
    print "oscilloscope started."
    return True

################
### SHUTDOWN ###
################

def stop():
    print "Stopping the logging scripts..."
    stop = subprocess.Popen([os.path.join(dirbin,'stoprun.sh')],stdout=subprocess.PIPE)
    print stop.communicate()[0]

def archive_run():
    print "Archiving this run's experimental data."
    arch_source = direxp
    arch_destination = os.path.join(dirarchive,tstart+'.tar.gz')
    print "source: %s\ndestination: %s\n" % (arch_source, arch_destination)
    archive = subprocess.Popen(['/bin/tar','-cvzf',arch_destination,arch_source,'--force-local'],
                               stderr=subprocess.PIPE)
    print "Archive process complete!\n standard error:%s" % (archive.communicate()[0])
    view_archive = subprocess.Popen(['/bin/ls','-lA',arch_destination], stdout=subprocess.PIPE)
    print "Here's the archive folder: \n%s" % (view_archive.communicate()[0])

def rm_run():
    import random
    delkey = "%.4f" % random.random()
    if raw_input("Are you absolutely certain you want to delete this run?\n type %s: " % (delkey)) == delkey:
        deletion = subprocess.Popen(['/bin/rm','-r',direxp])
        print "Everything deleted!\n stdout: %s\n stderr: %s\n" % (deletion.communicate())
    else:
        print "I guess you're not sure. Take your time...\n"
    
####################
### SYSTEM RESET ###
####################

'''
def reset_thermo():
    """ The camera is powered through a relay.  The relay coil is normally-closed,
    as controlled by a PNP BJT with the base tied to GPIO18.  """
    kill = sh.Command('killall')
    try:
        kill('thermograph')
    except:
        print("thermography not running; can't kill...")
    try:
        GPIO.output(18, GPIO.HIGH) ## turn the camera off
    except:
        setup_gpio()
    GPIO.output(18, GPIO.HIGH)
    time.sleep(2) ## wait for two seconds
    GPIO.output(18, GPIO.LOW) ## turn the camera on
    thermog()
    return True

def reset_thermometry():
    kill = sh.Command('killall')
    try:
        kill('temperature.sh','cat')
    except:
        pass
    thermom()
    return True

def reset_spectra():
    return True

def reset_oscilloscope():
    kill = sh.Command('killall')
    try:
        kill('oscilloscope.sh')
    except:
        pass
    oscillos()
    return True

################################
### MONITOR REALTIME LOGGING ###
################################

def sentinel(loops=1):
    loop = 0
    while (loop < loops):
        ntemp = sum([os.path.getsize(dtemp+f) for f in os.listdir(dtemp)])
        nthermo = len(os.listdir(dthermo))
        nspectra = len(os.listdir(dspectra))
        nosc = len(os.listdir(dosc))
        time.sleep(10)
        clear_output()
        print(datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
        ### monitor temperature by filesize - TESTED
        if ( ntemp == sum([os.path.getsize(dtemp+f) for f in os.listdir(dtemp)]) ):
            print("Thermometry has failed...")
            if (reset_temp()): print("Thermometry restarted.")
            else: print("Thermometry restart failed.")
        else: print("Thermometry is running.")
        ### monitor thermography by number of files - TESTED
        if ( nthermo == len(os.listdir(dthermo)) ):
            print("Thermography has failed...")
            if (reset_thermo()): print("Thermography restarted.")
            else: print("Thermography restart failed.")
        else: print("Thermography is running.")
        ### monitor spectroscopy by number of files - TESTED
        if ( nspectra == len(os.listdir(dspectra)) ):
            print("Spectroscopy has failed...")
            if (reset_spectra()): print("Spectroscopy restarted.")
            else: print("Spectroscopy restart failed.")
        else: print("Spectroscopy is running.")
        if ( nosc == len(os.listdir(dosc)) ):
            print("Oscilloscope has failed...")
            if (reset_oscilloscope()): print("Oscilloscope restarted.")
            else: print("Oscilloscope restart failed.")
        loop = loop + 1
    
def progressupdates(maxcycles,wait):
    cycles = 0
    while (cycles < maxcycles):
        progressbar((3,1,2))
        time.sleep(wait)

def progressbar(num):
    # see how things are REALLY going with the run
    f = 'temperaturehistory' #sorted(os.listdir(dtemp))
    data = numpy.genfromtxt(dtemp+f,delimiter=",",invalid_raise=False, skip_header=500000)
    print(dtemp+f)
    fig = plt.figure()
    ax = fig.add_subplot('111')
    colors = ['blue','red','green','purple','darkgrey']
    for position in range(0,len(num)):
        #print(num)
        #print(position)
        #print(num[position])
        if position == 1:
            ax2 = plt.twinx()
        #print([line[num[position]] for line in data])
        plt.plot([line[num[position]] for line in data],linestyle='None',color=colors[position],
                 marker='o',markersize=1,markeredgecolor=colors[position],zorder=position)
    ax.set_title('experimental run progress')
    ax.set_xlabel('counts, @100Hz',fontsize=12)
    ax.tick_params(axis='both',which='major',labelsize=10)
    ax.set_zorder(ax2.get_zorder()+1) # put ax in front of ax2 
    ax.patch.set_visible(False) # hide the 'canvas'
    ax2.set_ylim([15,40])
    fig.savefig(dexp+'progress.png',dpi=150)
    #fig.show()
    plt.close(fig)

def monitor(maxloops, delay):
    """here is a function description"""
    loops = 0
    while (loops < maxloops):

        runtime = (datetime.datetime.now()-started) #.strftime("%Y-%m-%d_%H:%M:%S")
        print("after running for " + str(runtime).split('.')[0] + ":")

        ### check to make sure logging is occurring
        if (len(os.listdir(dspectra)) < 3):
            print("WARNING: spectroscopy down")
        else:
            print(str(len(os.listdir(dspectra))) + " spectra")

        if (len(os.listdir(dthermo)) < 3):
            print("WARNING: thermography down")
        else:
            print(str(len(os.listdir(dthermo))) + " thermographs")

        # not happening on the Pi3 yet
        #if (len(os.listdir(dosc)) < 3):             
        #    print("WARNING: oscilloscope down") 
        #else:
        #    print(len(os.listdir(dosc)) + " oscilloscope traces so far!")

        if (len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0])) < 10):
            print("WARNING: thermometry down")
        else:
            print(str(len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0]))) + " temperature datapoints")

        print('')
        time.sleep(delay) # give it a few seconds to log some more stuff
        loops = loops + 1
'''

def start_experiment():
    mkdirs()
    time.sleep(2)
    #setup_gpio()
    capture_data()

def stop_experiment():
    stop()
    time.sleep(5)
    archive_run()
    time.sleep(5)
    rm_run()
    
def fail():
    stop()
    time.sleep(5)
    rm_run()

In [ ]:
start_experiment()

In [ ]:
#stop_experiment()
fail()
#stop()

In [2]:
mkdirs()


Setting up this experiment's directory structure...
Structure complete:
['out', 'bin', 'plot', 'in']
Experiment start time: 2016-06-28_02:10:15


In [ ]:
fa